<a href="https://colab.research.google.com/github/devac-yabuki/LLM-Learn/blob/main/llm-introduction/notebooks/chapter01.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 第1章 はじめに
## 1.1 transformers を使って自然言語処理を解いてみよう
- transformers をインストール
    - 大規模言語モデルをプログラムから扱うための標準的なライブラリ
    - オプション
        - ja
            - 日本語を扱う
        - sentencepiece
            - 分単位のバイト対符号化に使用
        - torch
            - Pytorch に関連するライブラリ

In [ ]:
!pip install transformers[ja,sentencepiece,torch]

- モデルを簡単に扱うための pipelines を使用

In [ ]:
from transformers import pipeline

### 1.1.1 文書分類
- テキストをあらかじめ定められたラベルに分解するタスク
- テキストから読み取れる感情を検出する文書分類を **感情分析** と呼ぶ


- 感情分析モデルを実行

In [ ]:
text_classification_pipeline = pipeline(
    model="llm-book/bert-base-japanese-v3-marc_ja"
)
positive_text = "世界には言葉がわからなくても感動する音楽がある。"

# positive_text の極性を予測
print(text_classification_pipeline(positive_text)[0])

In [ ]:
# negative_text の極性を予測
negative_text = "世界には言葉が出ないほどひどい音楽がある"
print(text_classification_pipeline(negative_text)[0])